In [1]:
%matplotlib inline

import matplotlib 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import json
import re

In [3]:
df_review = pd.read_json('review-sample-100k.json', lines = True)

In [146]:
rating = df_review['stars'].value_counts().sort_index()
rating_pct = 100.0*rating/sum(rating)
# list(rating_pct.index)
rating_pct.values

array([13.267,  8.309, 11.807, 23.897, 42.72 ])

In [4]:
df_review.head(10)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw
5,s2I_Ni76bjJNK9yG60iD-Q,0,2016-05-28,0,L_9BTb55X0GDtThi6GlZ6w,4,Had their chocolate almond croissant and it wa...,0,bv2nCi5Qv5vroFiqKGopiw
6,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-09-24,0,HRPm3vEZ_F-33TYVT7Pebw,5,Cycle Pub Las Vegas was a blast! Got a groupon...,1,_4iMDXbXZ1p1ONG297YEAQ
7,9_CGhHMz8698M9-PkVf0CQ,2,2012-05-11,0,ymAUG8DZfQcFTBSOiaNN4w,4,Who would have guess that you would be able to...,0,u0LXt3Uea_GidxRW1xcsfg
8,gkCorLgPyQLsptTHalL61g,0,2015-10-27,0,8UIishPUD92hXtScSga_gw,4,Always drove past this coffee house and wonder...,1,u0LXt3Uea_GidxRW1xcsfg
9,5r6-G9C4YLbC7Ziz57l3rQ,0,2013-02-09,0,w41ZS9shepfO3uEyhXEWuQ,3,"Not bad!! Love that there is a gluten-free, ve...",1,u0LXt3Uea_GidxRW1xcsfg


In [5]:
import spacy
nlp = spacy.load('en')

In [157]:
map(lambda a: len(list(nlp(a).sents)), df_review['text'].values[:10])


[6, 3, 8, 4, 6, 5, 8, 6, 5, 5]

In [9]:
## Removing al the \n in text
#df_review['text'] = df_review['text'].apply(lambda a: a.replace('\n',''))

#x = df_review['text'][:2]

## Trying to get sentence counts in spaCy
## tried to keep it simple, but doc.sents is a generator and does not have len()
import time
t0 = time.time()
# df_review['sent_count'] = df_review['text'].apply(lambda a: len(list(nlp(a).sents)))
sents_count_array = map(lambda a: len(list(nlp(a).sents)), df_review['text'].values)
t1 = time.time()
print("Use time in [min] {x}".format(x = (t1-t0)/60.0))


# ## code below works but slow...
# df_review['length_sent'] = 0

# #count = 0
# for i, text in enumerate(df_review['text'][:2]):
#     doc = nlp(text)
#     for s in doc.sents:
#         print s
    #print doc, '<<<the end>>>'
    #df_review['length_sent'][i] = 0
    #for sent in doc.sents:
        #df_review['length_sent'][i] += 1
    
#     count += 1
#     if count % 100 == 0:
#         print count



Use time in [min] 46.6888040344


In [10]:
df_review['sent_count'] = sents_count_array

In [161]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,sent_count
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,6
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,3
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw,8
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw,4
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw,6


In [171]:
# df_review.to_csv('yelp_review_cleaned',index = False, header = False)
df_review.to_pickle('yelp_review_cleaned.pkl')

In [175]:
df = pd.read_pickle("yelp_review_cleaned.pkl")
print df.shape
df.head()

(100000, 10)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id,sent_count
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,6
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,3
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw,8
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw,4
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw,6


In [6]:
import time
t0 = time.time()

count = 0

df_review['lemma'] = None
for i, text in enumerate(df_review['text']):
    count += 1
    #import pdb; pdb.set_trace()
    doc = nlp(text)
    df_review['lemma'][i] = map(lambda x: x.lemma_, doc)

    if count % 5000 == 0:
        print count

t1 = time.time()
print (t1-t0)/60
# lemma_array = map((lambda x:i.lemma_ for i in nlp(x)),(df_review['text'].values))
# y = df_review['text'][0:2]
# y = [i.lemma_ for i in nlp(y)]


# count = 0
# for i, text in enumerate(df_review['text']):
#     count += 1
#     #print text
#     doc = nlp(text)
#     df_review['lemma'][i] = []
#     for j in doc:
#         j = j.lemma_
#         df_review['lemma'][i] += [j]
    
#     if count%100 == 0:
#         print count
#     #print i.lemma_ for i in nlp(y)
    
#re.findall('place',y)

/opt/conda/envs/cp/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
370.364055149


In [7]:
import string
def rmv_punct(x):
    x = [''.join(c for c in s if c not in string.punctuation) for s in x]
    x = [i.replace('\n\n', '') for i in x]
    x = [s for s in x if s]
    return x
#y = df_review['lemma'][0]

In [8]:
df_review['lemma_clean'] = map(lambda a: rmv_punct(a),df_review['lemma'])

In [11]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,lemma,lemma_clean,sent_count
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,"[love, the, staff, ,, love, the, meat, ,, love...","[love, the, staff, love, the, meat, love, the,...",6
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,"[super, simple, place, but, amazing, nonethele...","[super, simple, place, but, amazing, nonethele...",3
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw,"[small, unassum, place, that, change, -PRON-, ...","[small, unassum, place, that, change, PRON, me...",8
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw,"[lester, 's, be, locate, in, a, beautiful, nei...","[lester, s, be, locate, in, a, beautiful, neig...",4
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw,"[love, come, here, ., yes, the, place, always,...","[love, come, here, yes, the, place, always, ne...",6


In [12]:
del df_review['lemma']

In [13]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,lemma_clean,sent_count
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,"[love, the, staff, love, the, meat, love, the,...",6
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,"[super, simple, place, but, amazing, nonethele...",3
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw,"[small, unassum, place, that, change, PRON, me...",8
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw,"[lester, s, be, locate, in, a, beautiful, neig...",4
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw,"[love, come, here, yes, the, place, always, ne...",6


In [14]:
df_review.to_pickle('yelp_review_cleaned_0710.pkl')

In [15]:
import pandas as pd
df = pd.read_pickle('yelp_review_cleaned_0710.pkl')

In [16]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,lemma_clean,sent_count
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,"[love, the, staff, love, the, meat, love, the,...",6
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,"[super, simple, place, but, amazing, nonethele...",3
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw,"[small, unassum, place, that, change, PRON, me...",8
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw,"[lester, s, be, locate, in, a, beautiful, neig...",4
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw,"[love, come, here, yes, the, place, always, ne...",6


In [131]:
print [w.lemma_ for w in nlp(u'carry carries carrying carried')]

[u'carry', u'carry', u'carry', u'carry']


In [138]:
xx = (u'carry carries carrying carried')
xx.lemma_

AttributeError: 'unicode' object has no attribute 'lemma_'